In [3]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from keras import Sequential,layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization
from keras.optimizers import SGD, Adam, RMSprop
import numpy as np

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

In [ ]:
data = ImageDataGenerator(rescale=1./255,
                          validation_split=0.25,  
                          rotation_range=40,      
                          width_shift_range=0.2,  
                          height_shift_range=0.2,
                          shear_range=10,         
                          zoom_range=0.2,         
                          horizontal_flip=True)   

data2 = ImageDataGenerator(rescale=1./255,
                           validation_split=0.25)

In [ ]:
train = data.flow_from_directory(
    '../input/cat-and-dog/training_set/training_set',
    batch_size=50, 
    target_size=(150,150), 
    subset='training') 
validation = data2.flow_from_directory(
    '../input/cat-and-dog/training_set/training_set',
    batch_size=50,
    target_size=(150,150),
    subset='validation') 

test = data2.flow_from_directory('../input/cat-and-dog/test_set/test_set',
                                    batch_size=50,
                                    target_size=(150,150))
                                    #class_mode='binary')

In [ ]:
train_imgs, labels = next(train)

In [ ]:
img=train_imgs[2]
print(labels[2])
plt.imshow(train_imgs[2])

In [ ]:
img1 = image.load_img('../input/cat-and-dog/training_set/training_set/dogs/dog.10.jpg',target_size=(150, 150))
img1 = image.img_to_array(img1)
img1 = img1.reshape((1,) + img1.shape)

i = 0
for batch in data.flow(img1):
    fig, axs = plt.subplots(1, 5, figsize=(5, 5))
    plt.figure(i,figsize=(2,2),)
    plt.xticks([])
    plt.yticks([])
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        
        break
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_imgs[i], cmap=plt.cm.binary)
    plt.xlabel(labels[np.argmax(labels[i])])
plt.show()


In [ ]:
train.class_indices 

In [ ]:
class_names = list(train.class_indices.keys())
print(class_names)

In [ ]:
cnn=Sequential()
cnn.add(Conv2D(filters=32, 
                 kernel_size=(5,5),  
                 padding='same',  
                 activation='relu',  
                 input_shape=(150, 150, 3)))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Conv2D(64, (3,3), activation='relu'))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Conv2D(128, (3,3), activation='relu'))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Conv2D(128, (3,3), activation='relu'))
cnn.add(MaxPooling2D(strides=2))
cnn.add(Flatten())
cnn.add(Dense(512, activation='relu'))
cnn.add(Dense(2, activation='softmax'))
cnn.summary()

In [ ]:
cnn.compile(optimizer='rmsprop', 
            loss='categorical_crossentropy', 
            metrics=['acc'])

In [ ]:
history = cnn.fit_generator(train,steps_per_epoch=120,epochs=30,
                  validation_data=validation, 
                  validation_steps = 40)

In [ ]:
loss, acc=cnn.evaluate(test, steps=40)
print('Accuracy:', acc)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    true_label1 = np.argmax(true_label)
    if predicted_label == true_label1:
        color = 'green'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                         100*np.max(predictions_array),
                                         class_names[true_label1]),
                                         color=color)

def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    true_label1 = np.argmax(true_label)
    plt.grid(False)
    plt.xticks(range(2))
    plt.yticks([])
    thisplot = plt.bar(range(2), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label1].set_color('green')

In [ ]:
test_imgs, test_labels = next(validation)
predictions = cnn.predict(test_imgs)

In [ ]:
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_imgs)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

In [2]:
def to_EMA(point,a=0.3):
    ret=[]
    EMA=point[0]
    for pt in points:
        EMA=pt*a+EMA*(1-a)
        ret.append(EMA)
    return ret